In [ ]:
import pandas as pd

In [ ]:
#1 SQL)
SELECT actor_id, director_id
FROM ActorDirector
GROUP BY actor_id, director_id
HAVING count(*)>=3

#Pandas
result = (
    ActorDirector.groupby(['actor_id', 'director_id'])
      .size()
      .reset_index(name='count')
      .query('count >= 3')
      [['actor_id', 'director_id']]
)


In [ ]:
#2 SQL)
SELECT
    user_id,
    CONCAT(UPPER(SUBSTRING(name, 1,1)), LOWER(SUBSTRING(name, 2))) AS name
FROM
    Users
ORDER BY
    user_id;

#Pandas
result = (
    Users.assign(
        name = df['name'].str[0].str.upper() + df['name'].str[1:].str.lower()
        )
    .sort_values('user_id')
)

In [ ]:
#3 SQL)
SELECT Person.firstName, Person.lastName, Address.city, Address.state
FROM
    Person LEFT JOIN Address ON Person.personId = Address.personId

#Pandas
result = (
    person_df
    .merge(address_df, on='personId', how='left')
    [['firstName', 'lastName', 'city', 'state']]
)


In [ ]:
#4 SQL)
SELECT
    MAX(salary) AS SecondHighestSalary
FROM
    Employee
WHERE
    salary < (SELECT MAX(salary) FROM Employee)

#Pandas
max_salary = Employee['salary'].max()
second_highest_salary = Employee.loc[Employee['salary'] < max_salary, 'salary'].max()


In [ ]:
#5 SQL)
SELECT p.product_name,
    sum(o.unit) AS unit
FROM Products p
JOIN Orders o ON p.product_id = o.product_id
WHERE o.order_date LIKE '2020-02%'
GROUP BY p.product_name HAVING sum(o.unit)>= 100

#Pandas
merged = products_df.merge(orders_df, on='product_id', how='inner')

result = (
    merged[merged['order_date'].str.startswith('2020-02')]
    .groupby('product_name', as_index=False)['unit'].sum()
    .query('unit >= 100')
)

In [ ]:
#6 SQL)
SELECT EmployeeUNI.unique_id, Employees.name
FROM
    Employees LEFT JOIN EmployeeUNI ON EmployeeUNI.id = Employees.id

#Pandas
merged = Employees_df.merge(EmployeeUNI_df, on='id', how='left')
result = merged[['unique_id', 'name']]


In [ ]:
#7 SQL)
WITH game AS(
SELECT
    player_id, min(event_date) as event_start_date
from
    Activity
group by player_id)
SELECT
    ROUND((count(distinct activity.player_id)/(select count(distinct activity.player_id) from activity)),2) AS fraction
FROM game
JOIN Activity on Activity.player_id = game.player_id
WHERE datediff(event_start_date, event_date) = -1

#Pandas
game = (
    activity.groupby('player_id', as_index=False)['event_date']
    .min()
    .rename(columns={'event_date': 'event_start_date'})
)
merged = activity.merge(game, on='player_id', how='inner')
next_day = merged[merged['event_date'] - merged['event_start_date'] == pd.Timedelta(days=1)]
fraction = round(next_day['player_id'].nunique() / activity['player_id'].nunique(), 2)
print(fraction)

In [ ]:
#8 SQL)
SELECT
    Project.project_id,
    round((sum(Employee.experience_years))/(count(Employee.employee_id)),2) AS average_years
FROM
    Project LEFT JOIN Employee on Project.employee_id = Employee.employee_id
GROUP BY project_id

#Pandas
merged=Employee.merge(Project, on= 'employee_id', how='left')
result = (
    merged.groupby('project_id', as_index=False)
    .agg(average_years=('experience_years', 'mean'))
)

result['average_years'] = result['average_years'].round(2)

In [ ]:
#9 SQL)
WITH ranked_employees AS (
    SELECT
        e.name AS employee,
        e.salary AS salary,
        d.name AS department,
        DENSE_RANK() OVER(
            PARTITION BY d.name
            ORDER BY e.salary DESC
        ) AS salary_ranking
    FROM
        Employee e
        JOIN Department d ON e.departmentId = d.id
)
SELECT
    employee,
    salary,
    department
FROM ranked_employees
WHERE salary_ranking <= 3

#Pandas
merged = Employee.merge(Department, left_on='departmentId', right_on='id', how='inner')
merged['salary_ranking'] = (
    merged.groupby('Department.name')['salary']
          .rank(method='dense', ascending=False)
)
top_earners = merged[merged['salary_ranking'] <= 3]
higher_earner = top_earners[['Department.name', 'Employee.name', 'salary']].rename(
    columns={'Department.name': 'Department', 'Employee.name': 'Employee'}
)
print(higher_earner)
